In [34]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

def odds_conv(odds):
    """
    Convert odds to implied probabilities.
    """
    if odds > 0:
        return 1 / (odds / 100 + 1)
    else:
        return 1 / (1 + 100 / abs(odds))

def fair_odds(odds_a, odds_b):
    """
    Calculate fair odds for two events.
    """
    fair_a = odds_conv(odds_a) / (odds_conv(odds_a) + odds_conv(odds_b))
    fair_b = odds_conv(odds_b) / (odds_conv(odds_a) + odds_conv(odds_b))
    return (fair_a, fair_b)

def avg(x, y):
    """
    Calculate the average of two numbers.
    """
    return (x + y) / 2

def profit(stake, odds):
    """
    Calculate profit from a bet.
    """
    if odds > 0:
        return odds / 100 * stake
    else:
        return 100 / abs(odds) * stake

def kelly_criterion(b, p, q):
    """
    Calculate the Kelly Criterion for optimal betting.
    """
    return (b * p - q) / b

def width(a, b):
    """
    Calculate the width between two numbers.
    """
    if a < 0 and b < 0:
        return abs(int(str(a)[-2:]) + int(str(b)[-2:]))
    else:
        return abs(int(str(a)[-2:]) - int(str(b)[-2:]))

def get_sport_odds(api_key, sport_key='upcoming', regions='us', markets='h2h', oddsFormat='american'):
    """
    Fetches sports odds data from the Odds API.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
        'oddsFormat': oddsFormat,
        'markets': markets,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data, markets):
    """
    Processes the odds data and creates a DataFrame.
    """
    data_list = []

    if markets == 'h2h':
        for game in data:
            game_date = game['commence_time']
            sport = game['sport_title']
            home_team = game['home_team']
            away_team = game['away_team']
            for bookmaker in game['bookmakers']:
                bookmaker_name = bookmaker['title']
                for market in bookmaker['markets']:
                    if market['key'] == 'h2h':
                        for outcome in market['outcomes']:
                            team_name = outcome['name']
                            if team_name.lower() == home_team.lower():
                                team_type = 'home'
                            elif team_name.lower() == away_team.lower():
                                team_type = 'away'
                            else:
                                team_type = 'unknown'
                            row = {
                                'date': game_date,
                                'sport': sport,
                                'home_team': home_team,
                                'away_team': away_team,
                                'bookmaker': bookmaker_name,
                                'team_type': team_type,
                                'price': outcome['price']
                            }
                            data_list.append(row)
        df = pd.DataFrame(data_list)
        pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
        pivoted_df.reset_index(inplace=True)
        pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
        pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
        pivoted_df.fillna('N/A', inplace=True)

    elif markets == 'spreads':
        for game in data:
            game_date = game['commence_time']
            sport_key = game['sport_key']
            home_team = game['home_team']
            away_team = game['away_team']
            for bookmaker in game['bookmakers']:
                bookmaker_name = bookmaker['title']
                for market in bookmaker['markets']:
                    if market['key'] == 'spreads':
                        for outcome in market.get('outcomes', []):
                            team_name = outcome['name']
                            spread = outcome.get('point', 'N/A')
                            price = outcome.get('price', 'N/A')
                            if team_name.lower() == home_team.lower():
                                team_type = 'home'
                            elif team_name.lower() == away_team.lower():
                                team_type = 'away'
                            else:
                                team_type = 'unknown'

                            data_list.append({
                                'date': game_date,
                                'sport': sport_key,
                                'home_team': home_team,
                                'away_team': away_team,
                                'team_type': team_type,
                                'bookmaker': bookmaker_name,
                                'spread': spread,
                                'price': price
                            })
        df = pd.DataFrame(data_list)
        pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type','spread'],
                            columns='bookmaker',
                            values='price',
                            aggfunc='first')

        pivoted_df.reset_index(inplace=True)
        pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
        pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
        pivoted_df.fillna('N/A', inplace=True)

    return pivoted_df

def calculate_ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.
    """
    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_a = width(pin[0], pin[1])
        return 0, a_ev, dk[0], bet, width_a
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_b = width(pin[0], pin[1])
        return 1, b_ev, dk[1], bet, width_b

SPORTBOOK = 'FanDuel'

def main(api_key, sport_key, regions, markets, odds_format, stake, bankroll):
    """
    Main function to fetch, process, and analyze sports odds data.
    """
    data = get_sport_odds(api_key, sport_key=sport_key, regions=regions, markets=markets, oddsFormat=odds_format)
    if markets == 'h2h':
        pivoted_data = process_odds_data(data, markets=markets)
        test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'team', SPORTBOOK, 'BetOnline.ag', 'Bovada']]
        test['matchup'] = test['home_team'] + ' vs ' + test['away_team']
        test['team_x'] = np.where(test['home_team'] == test['team'], 'home', 'away')
        test = test.sort_values(['matchup', 'team_x'], ascending=False)

        test['check'] = np.where((test[SPORTBOOK] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'), 1, 0)
        test = test[test['check'] == 0]
        
        cols = [SPORTBOOK, 'BetOnline.ag', 'Bovada']
        for col in cols:
            test[col] = test[col].astype('int')

        consolidated_df = test.groupby(['date', 'sport', 'home_team', 'away_team']).agg({
            SPORTBOOK: tuple,
            'BetOnline.ag': tuple,
            'Bovada': tuple
        }).reset_index()

        result_df = consolidated_df.apply(lambda row: calculate_ev(row[SPORTBOOK], row['BetOnline.ag'], row['Bovada'], stake, bankroll), axis=1, result_type='expand')

        result_df.columns = ['team_index', 'ev', 'odds', 'bet', 'width']
        final_df = pd.concat([consolidated_df, result_df], axis=1)

        final_df['team'] = np.where(final_df['team_index'] == 0, final_df['home_team'], final_df['away_team'])
        final_df = final_df.sort_values('ev', ascending=False)
    
    elif markets == 'spreads':

        pivoted_data = process_odds_data(data, markets=markets)
        test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'team', 'spread', SPORTBOOK, 'BetOnline.ag', 'Bovada']]
        test['matchup'] = test['home_team'] + ' vs ' + test['away_team']
        test['team_x'] = np.where(test['home_team'] == test['team'], 'home', 'away')
        test = test.sort_values(['matchup', 'team_x'], ascending=False)

        test['check'] = np.where((test[SPORTBOOK] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'), 1, 0)
        test = test[test['check'] == 0]
        cols = [SPORTBOOK, 'BetOnline.ag', 'Bovada']
        for col in cols:
            test[col] = test[col].astype('int')
        test['spread'] = abs(test['spread'])
        consolidated_df = test.groupby(['date', 'sport', 'home_team', 'away_team', 'spread']).agg({
            SPORTBOOK: tuple,
            'BetOnline.ag': tuple,
            'Bovada': tuple
        }).reset_index()


        result_df = consolidated_df.apply(lambda row: calculate_ev(row[SPORTBOOK], row['BetOnline.ag'], row['Bovada'], stake, bankroll), axis=1, result_type='expand')

        result_df.columns = ['team_index', 'ev', 'odds', 'bet', 'width']
        final_df = pd.concat([consolidated_df, result_df], axis=1)

        final_df['team'] = np.where(final_df['team_index'] == 0, final_df['home_team'], final_df['away_team'])
        final_df = final_df.sort_values('ev', ascending=False)

    return final_df

In [35]:
result_df = main(api_key='287f26ecf5b58ab73781cfaf78310c08', sport_key='upcoming', regions='us', odds_format='american', markets='spreads', stake=100, bankroll=1000).sort_values("ev", ascending=False)
result_df.head(10)

,date,sport,home_team,away_team,spread,FanDuel,BetOnline.ag,Bovada,team_index,ev,odds,bet,width,team
2,2023-10-10T01:07:00Z,baseball_mlb,Los Angeles Dodgers,Arizona Diamondbacks,1.5,"(134, -162)","(130, -149)","(130, -150)",0.0,-1.604237,134.0,-182.629313,19.0,Los Angeles Dodgers
0,2023-10-09T23:30:00Z,basketball_nba_preseason,New York Knicks,Boston Celtics,8.5,"(-110, -110)","(-115, -105)","(-105, -115)",1.0,-4.545455,-110.0,0.000000,20.0,Boston Celtics
1,2023-10-10T01:00:00Z,basketball_nba_preseason,Los Angeles Lakers,Brooklyn Nets,4.5,"(-110, -110)","(-110, -110)","(-110, -110)",1.0,-4.545455,-110.0,0.000000,20.0,Brooklyn Nets


In [37]:
result_df = main(api_key='287f26ecf5b58ab73781cfaf78310c08', sport_key='upcoming', regions='us', odds_format='american', markets='h2h', stake=100, bankroll=1000).sort_values("ev", ascending=False)
result_df.head()

,date,sport,home_team,away_team,FanDuel,BetOnline.ag,Bovada,team_index,ev,odds,bet,width,team
1,2023-10-09T23:00:00Z,Brazil Série B,Sport Recife,Ponte Preta,"(-370, 410, 1000)","(-333, 410, 670)","(-340, 440, 1075)",0.0,1.844935,-370.0,601.187951,23.0,Sport Recife
2,2023-10-09T23:30:00Z,NBA Preseason,New York Knicks,Boston Celtics,"(-355, 270)","(-303, 255)","(-300, 250)",1.0,1.455475,270.0,485.439638,52.0,Boston Celtics
0,2023-10-09T22:07:00Z,MLB,Atlanta Braves,Philadelphia Phillies,"(-112, -112)","(-149, 138)","(-105, -125)",0.0,1.001366,-112.0,0.000000,11.0,Atlanta Braves
5,2023-10-10T01:00:00Z,NBA Preseason,Los Angeles Lakers,Brooklyn Nets,"(-200, 160)","(-179, 160)","(-185, 160)",1.0,-2.908049,160.0,268.292683,19.0,Brooklyn Nets
4,2023-10-10T00:18:00Z,NFL,Las Vegas Raiders,Green Bay Packers,"(-132, 112)","(-132, 111)","(-130, 110)",1.0,-3.360322,112.0,93.466708,21.0,Green Bay Packers
